In [2]:
import os
import gc
import re
import json
import pickle
import datetime
from tqdm import tqdm
from typing import Union

import numpy as np
import pandas as pd
pd.options.display.max_columns = 50

from typing import Union

import seaborn
import matplotlib.pyplot as plt
plt.style.use("seaborn-darkgrid")

from scipy.stats import linregress

from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

# custom funcs
from script import WRMSSEEvaluator
from script import cache_result
from script import reduce_mem_usage
from script import load_pickle, dump_pickle

In [57]:
train = pd.read_pickle('features/melted_and_merged_train.pkl')

In [58]:
train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,quarter,week,weekofyear,day,dayofweek,is_year_end,is_year_start,is_quarter_end,is_quarter_start,is_month_end,is_month_start,sell_price
7,HOBBIES_1_008_CA_1_validation,1444,3,1,0,0,d_1,12,2011-01-29,11101,Saturday,1,1,2011,30,4,4,2,0,0,0,1,4,4,29,5,False,False,False,False,False,False,0.459961
8,HOBBIES_1_009_CA_1_validation,1445,3,1,0,0,d_1,2,2011-01-29,11101,Saturday,1,1,2011,30,4,4,2,0,0,0,1,4,4,29,5,False,False,False,False,False,False,1.559570
9,HOBBIES_1_010_CA_1_validation,1446,3,1,0,0,d_1,0,2011-01-29,11101,Saturday,1,1,2011,30,4,4,2,0,0,0,1,4,4,29,5,False,False,False,False,False,False,3.169922
11,HOBBIES_1_012_CA_1_validation,1448,3,1,0,0,d_1,0,2011-01-29,11101,Saturday,1,1,2011,30,4,4,2,0,0,0,1,4,4,29,5,False,False,False,False,False,False,5.980469
14,HOBBIES_1_015_CA_1_validation,1451,3,1,0,0,d_1,4,2011-01-29,11101,Saturday,1,1,2011,30,4,4,2,0,0,0,1,4,4,29,5,False,False,False,False,False,False,0.700195


In [59]:
train = train[['id', 'd', 'sales', 'sell_price']]

In [60]:
# this is scaaaaaaaaaale !!!!!!!

def est_scale(x):
    series = x.shift(28).values
    series = series[~np.isnan(series)][np.argmax(series != 0):]
    if len(series) > 0:
        scale = ((series[1:] - series[:-1]) ** 2).mean()
    else:
        scale = 1
    return scale

train.groupby('id')['sales'].transform(lambda x: est_scale(x)).tail()

60034805    1.323711
60034806    1.034021
60034807    2.283505
60034808    2.123752
60034809    6.519065
Name: sales, dtype: float64

weight はできなかったので、明日のあさおきたらやろう。

In [69]:
# あとは全体からの売上占有率を計算したい。
train['total_sales_price'] = train[['sales', 'sell_price']].prod(axis=1)
train['ahead_total_sales_price'] = train.groupby('id')['total_sales_price'].transform(lambda x: x.rolling(28).sum())
train['overall_d_total_sales_price'] = train.groupby('d')['total_sales_price'].transform('sum')

In [70]:
# train['weight'] = train['ahead_total_sales_price'] / train['overall_ahead_total_sales_price']

In [71]:
train[~train['ahead_total_sales_price'].isnull()].tail()

,id,d,sales,sell_price,total_sales_price,ahead_total_sales_price,overall_ahead_total_sales_price,weight,overall_d_total_sales_price
60034805,FOODS_3_823_WI_3_validation,d_1969,0,2.980469,0.0,0.0,0.0,NaN,0.0
60034806,FOODS_3_824_WI_3_validation,d_1969,0,2.480469,0.0,0.0,0.0,NaN,0.0
60034807,FOODS_3_825_WI_3_validation,d_1969,0,3.980469,0.0,0.0,0.0,NaN,0.0
60034808,FOODS_3_826_WI_3_validation,d_1969,0,1.280273,0.0,0.0,0.0,NaN,0.0
60034809,FOODS_3_827_WI_3_validation,d_1969,0,1.000000,0.0,0.0,0.0,NaN,0.0
